# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [19]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


In [18]:
adult_dt

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [3]:
from sklearn.model_selection import train_test_split
X = adult_dt.drop(columns="income") 
Y = adult_dt['income']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

+ The random_state in train_test_split seeds the random number generator, making the data split reproducible. Setting random_state=42 means the train-test split will be the same each time the code runs.
  
+ Reproducibility is crucial in data science as it ensures consistent results. With a fixed random_state, we get the same data split every time, allowing fair model comparison, easier debugging, and reliable sharing of results.



# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [20]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

numerical_features = [
    "age", "fnlwgt", "education-num", "capital-gain", 
    "capital-loss", "hours-per-week"
]
categorical_features = [
    "workclass", "education", "marital-status", "occupation", 
    "relationship", "race", "sex", "native-country"
]

numerical_transformer = Pipeline(steps=[
    ("knn_imputer", KNNImputer(n_neighbors=7, weights="distance")),
    ("scaler", RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ("simple_imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore", drop="if_binary"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num_transforms", numerical_transformer, numerical_features),
        ("cat_transforms", categorical_transformer, categorical_features)
    ]
)

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [21]:

from sklearn.ensemble import RandomForestClassifier

model_pipeline = Pipeline(steps=[
    ("preprocessing", preprocessor),    
    ("classifier", RandomForestClassifier())  
])

model_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('knn_imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('simple_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [23]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score

scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

cv_results = cross_validate(
    model_pipeline, 
    X_train, 
    Y_train, 
    cv=5, 
    scoring=scoring, 
    return_train_score=True
)

print("Cross-validation results:")
for metric in scoring.keys():
    train_scores = cv_results[f'train_{metric}']
    test_scores = cv_results[f'test_{metric}']
    
    print(f"\nMetric: {metric}")
    print(f"  Training scores (per fold): {train_scores}")
    print(f"  Validation scores (per fold): {test_scores}")
    print(f"  Average training score: {train_scores.mean():.4f}")
    print(f"  Average validation score: {test_scores.mean():.4f}")

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Cross-validation results:

Metric: neg_log_loss
  Training scores (per fold): [-0.08122077 -0.08113766 -0.08119539 -0.08209184 -0.08131713]
  Validation scores (per fold): [-0.35033503 -0.37286959 -0.37422037 -0.35711547 -0.36702815]
  Average training score: -0.0814
  Average validation score: -0.3643

Metric: roc_auc
  Training scores (per fold): [1. 1. 1. 1. 1.]
  Validation scores (per fold): [0.90481886 0.90280237 0.90277115 0.90715451 0.90228   ]
  Average training score: 1.0000
  Average validation score: 0.9040

Metric: accuracy
  Training scores (per fold): [1. 1. 1. 1. 1.]
  Validation scores (per fold): [0.84974775 0.84821233 0.85256692 0.86002633 0.85849057]
  Average training score: 1.0000
  Average validation score: 0.8538

Metric: balanced_accuracy
  Training scores (per fold): [1. 1. 1. 1. 1.]
  Validation scores (per fold): [0.77482693 0.76920415 0.77408012 0.78331099 0.78014357]
  Average training score: 1.0000
  Average validation score: 0.7763


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [25]:
results_df = pd.DataFrame(cv_results)
sorted_results_df = results_df.sort_values(by='test_neg_log_loss')
sorted_results_df

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy
2,8.801750,0.195881,-0.374220,-0.081195,0.902771,1.0,0.852567,1.0,0.774080,1.0
1,8.819343,0.197052,-0.372870,-0.081138,0.902802,1.0,0.848212,1.0,0.769204,1.0
4,9.098544,0.197317,-0.367028,-0.081317,0.902280,1.0,0.858491,1.0,0.780144,1.0
3,9.002830,0.211997,-0.357115,-0.082092,0.907155,1.0,0.860026,1.0,0.783311,1.0
0,8.982736,0.198607,-0.350335,-0.081221,0.904819,1.0,0.849748,1.0,0.774827,1.0


Calculate the mean of each metric. 

In [26]:
mean_results = sorted_results_df.mean()

print("Mean results for each metric:")
print(mean_results)

Mean results for each metric:
fit_time                   8.941041
score_time                 0.200171
test_neg_log_loss         -0.364314
train_neg_log_loss        -0.081393
test_roc_auc               0.903965
train_roc_auc              1.000000
test_accuracy              0.853809
train_accuracy             1.000000
test_balanced_accuracy     0.776313
train_balanced_accuracy    1.000000
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [27]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

model_pipeline.fit(X_train, Y_train)

Y_pred = model_pipeline.predict(X_test)
Y_pred_proba = model_pipeline.predict_proba(X_test)

test_metrics = {
    "neg_log_loss": log_loss(Y_test, Y_pred_proba),
    "roc_auc": roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    "accuracy": accuracy_score(Y_test, Y_pred),
    "balanced_accuracy": balanced_accuracy_score(Y_test, Y_pred)
}

test_metrics

{'neg_log_loss': 0.3699160555247284,
 'roc_auc': 0.9012164603872153,
 'accuracy': 0.8551540587572934,
 'balanced_accuracy': 0.7753121726614367}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

Recoding income as a binary variable (0/1) from the start is convenient because binary encoding is required for classification models and evaluation metrics, such as ROC AUC and log loss.



## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.